# Quick test over the trained model

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import pandas as pd
import random
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


In [3]:
from torchvision import models

# Cargar arquitectura
model = models.mobilenet_v3_large(pretrained=False)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 2)
model.load_state_dict(torch.load('../../model/mobilenetv3_cats_vs_dogs.pth', map_location=device))
model = model.to(device)
model.eval()


/home/juancarlos/Documents/dev/vision/cats_vs_dogs_parallel_inference/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/juancarlos/Documents/dev/vision/cats_vs_dogs_parallel_inference/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [4]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
df = pd.read_csv('../../csv/training_images.csv')
test_df = df[df['type'] == 'test']

In [6]:
sample_df = test_df.sample(n=100, random_state=42).reset_index(drop=True)

In [7]:
# Inference
correct = 0

class_to_idx = {'cat': 0, 'dog': 1}

for idx, row in sample_df.iterrows():
    img_path = row['image']
    true_label = class_to_idx[row['class']]

    # Cargar imagen
    image = Image.open(img_path).convert('RGB')
    image = test_transform(image)
    image = image.unsqueeze(0).to(device)  # batch size 1

    # Predicción
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    is_correct = (predicted.item() == true_label)
    correct += int(is_correct)
    
    print(f"[{idx+1}] Predicted: {'cat' if predicted.item() == 0 else 'dog'}, True: {row['class']}, Correct: {is_correct}")


[1] Predicted: cat, True: cat, Correct: True
[2] Predicted: dog, True: dog, Correct: True
[3] Predicted: cat, True: cat, Correct: True
[4] Predicted: dog, True: dog, Correct: True
[5] Predicted: cat, True: cat, Correct: True
[6] Predicted: dog, True: dog, Correct: True
[7] Predicted: dog, True: dog, Correct: True
[8] Predicted: cat, True: cat, Correct: True
[9] Predicted: dog, True: dog, Correct: True
[10] Predicted: dog, True: dog, Correct: True
[11] Predicted: cat, True: cat, Correct: True
[12] Predicted: cat, True: cat, Correct: True
[13] Predicted: cat, True: cat, Correct: True
[14] Predicted: cat, True: cat, Correct: True
[15] Predicted: cat, True: cat, Correct: True
[16] Predicted: cat, True: cat, Correct: True
[17] Predicted: dog, True: dog, Correct: True
[18] Predicted: dog, True: dog, Correct: True
[19] Predicted: dog, True: dog, Correct: True
[20] Predicted: cat, True: cat, Correct: True
[21] Predicted: dog, True: dog, Correct: True
[22] Predicted: cat, True: cat, Correct: Tr

In [10]:
# --- 📊 Resultado final ---
accuracy = correct / 100
print(f"\n✅ Accuracy on the 100 samples from the random mass inference test set: {accuracy*100:.2f}%")


✅ Accuracy on the 100 samples from the random mass inference test set: 96.00%
